In [11]:
import geemap
from collection_config import veg_indices
from ipyleaflet import WidgetControl, Marker, DrawControl, Icon, Polygon, GeoJSON
from ipywidgets import Dropdown, Button, Output, Accordion, Layout, Label, VBox
import ee
from bqplot import pyplot as plt
from datetime import datetime
from geopy.geocoders import Nominatim
import shapely
import json
import geojson
import copy
import numpy

## Cquest Earth App

In [12]:
m = geemap.Map(center=(52.6, 13.4), 
                 zoom=4,
                 add_google_map = False,
                 layer_ctrl = False,
                 fullscreen_ctrl = True,            
                 data_ctrl = False,
                 zoom_ctrl = True,
                 draw_ctrl = False,
                 search_ctrl = False,
                 measure_ctrl = False,
                 scale_ctrl = False,
                 toolbar_ctrl = False,
                 attribution_ctrl= False
              )

In [13]:
#v0.1.0: Map with different layers of datasets
#geemap: addLayer(), ipyleaflet: add_layer() -> both work
for dataset in veg_indices:
    m.addLayer(veg_indices[dataset]['visual'], 
               veg_indices[dataset]['vis_params'],
               dataset, True, 0)
m.layer_opacity('MODIS NDVI (250m)', 0.7)

In [14]:
#v0.2.0: LAYER SELECTION
#Options list is the list that holds options of layers selection box
options_list = []
for layer in m.layers:
    options_list.append(layer.name)
options_list.remove(options_list[0]) #remove the first layer's name from the list because it is the basemap layer

# Adds a Dropdown widget for layers selection
layers_selection = Dropdown(
    options=options_list,
    description='Dataset'
)

# Handles Dropdown control event
def on_click(change):
    old_layer_name = change['old']
    new_layer_name = change['new']
    m.layer_opacity(old_layer_name, 0)
    m.layer_opacity(new_layer_name, 0.7)
#ipywidgets Dropdown function observe()
layers_selection.observe(on_click, 'value')
# basemap_control = WidgetControl(widget=layers_selection, position='topleft')

In [15]:
#necessary variables for drawing geometries on the map
geom_list = []  #List of geometries on the map
ee_object_list = []  #List of geometries as ee object
####
icon = Icon(icon_url="https://img.icons8.com/ultraviolet/40/000000/marker.png", 
            icon_size = [40, 40], 
            icon_anchor = [20, 40])

In [16]:
#v0.3.0: DRAWING GEOMETRIES ON THE MAP
#Drawing geometries on the map
dc = DrawControl(marker={'shapeOptions': {'iconUrl':"https://img.icons8.com/office/80/000000/marker.png"}},
                 polygon={'shapeOptions': {'color': '#19cdff'}},
                 rectangle={},
                 circle={},
                 circlemarker={},
                 polyline={},
                 remove=False,
                 edit=False)

def handle_draw(target, action, geo_json):
    drawType = geo_json['geometry']['type']
    latlon = geo_json['geometry']['coordinates']
    #Handle point event
    if drawType == 'Point':
        point = ee.Geometry.Point(latlon)
        ee_object_list.append(point)
        coordinates = copy.deepcopy(latlon)
        coordinates.reverse()
        location = locator.reverse(coordinates)
        marker = Marker(location = coordinates, 
                        icon = icon, 
                        draggable = False, 
                        title = location.address)
        geom_list.append(marker)
        m.add_layer(marker)
    #Handle polygon event
    elif drawType == 'Polygon':
        ee_polygon = ee.Geometry.Polygon(latlon)
        ee_object_list.append(ee_polygon)
        coordinates = copy.deepcopy(latlon[0])
        for coord in coordinates:
            coord.reverse()
        polygon = Polygon(locations = coordinates, color = '#19cdff', fillColor = 'blue')
        geom_list.append(polygon)
        m.add_layer(polygon)  
    #clear drawing features
    dc.clear()
    #condition to limit the number of geometries
    if len(geom_list) == 3:
        m.remove_layer(geom_list[0])
        del geom_list[0]
    if len(ee_object_list) ==3:
        del ee_object_list[0]
#ipyleaflet function on_draw() to handle drawing event
dc.on_draw(handle_draw)

In [17]:
#delete all drawing features on the map
reset_button = Button(icon='trash', 
                      disabled=False,
                      tooltip='Delete all geometries, plots and outputs',
                      layout=Layout(width='30px',height='30px')
                     )
reset_control = WidgetControl(widget=reset_button, 
                              position='topleft')
def on_button_clicked(b):
    plt.clear()
    plot_widget.clear_output()
    [m.remove_layer(layer) for layer in geom_list]
    geom_list.clear()
    ee_object_list.clear()
reset_button.on_click(on_button_clicked)

In [18]:
#remove NONE from dict function
def drop_null_from_dict(ts_aoi):
    filtered = {k: v for k, v in ts_aoi.items() if v is not None}
    ts_aoi.clear()
    ts_aoi.update(filtered)
    return ts_aoi

#convert timestamps
def timeStampsToString(timestamps):
    return ee.Date(timestamps).format("YYYY-MM-dd")

In [19]:
#v0.4.0 & v0.5.0: FETCH DATA FROM GEOMETRIES AND PLOT TIME-SERIES
options_list2 = ['...','MODIS NDVI (250m)','MODIS EVI (250m)','MODIS GPP (500m)','MODIS NPP (500m)','MODIS LAI (500m)','LANDSAT 7 NDVI (30m)','LANDSAT 7 EVI (30m)']
chart_selection =  Dropdown(
    options=options_list2,
    description="Dataset"
)

#"Output"-box to display chart
plot_widget = Output(layout={'border':'1px solid black'})
plot_output = WidgetControl(widget=plot_widget,
                            position='topright')

#Handle select event
locator = Nominatim(user_agent="myGeocoder")
legend = ['\U0001F535', '\U0001F534']
def on_chart_clicked(change):
    with plot_widget:
        try:
            plt.clear()
            plot_widget.clear_output()
            print("LOADING . . .")
            chart_option=change['new']
            collection = veg_indices[chart_option]['collection']
            resolution = veg_indices[chart_option]['resolution']
            band = veg_indices[chart_option]['band']
            timestamps = collection.aggregate_array("system:time_start")
            date_strings = timestamps.map(timeStampsToString)
            stacked_image = collection.select(band).toBands().rename(date_strings)
            title = chart_option + ' Time-Series'
            plt.figure(1, title = title)
            for geom in ee_object_list:
                ts_aoi = ee.Image(stacked_image).reduceRegion(reducer=ee.Reducer.mean(),
                                                              geometry = geom,
                                                              scale = resolution,
                                                              bestEffort = True).getInfo()
                ts_aoi = drop_null_from_dict(ts_aoi)
                xValues = numpy.asarray(list(ts_aoi.keys()), dtype = 'datetime64[D]')
                yValues = numpy.asarray(list(ts_aoi.values()))
                if ee_object_list.index(geom) == 0:
                    plt.plot(xValues, yValues, '-b')
                if ee_object_list.index(geom) == 1:
                    plt.plot(xValues, yValues, '-r')
            plt.ylabel(label=veg_indices[chart_option]['unit'])
            plot_widget.clear_output()
            plt.show()
        except Exception as e:
            plot_widget.clear_output()
            print("Failed to create plot :(")
            raise Exception(e)
        for geom, legend_symbol in zip(geom_list, legend):
            place_name = None
            if type(geom) == Polygon:
                poly_coords = geom.locations
                poly = shapely.geometry.Polygon(poly_coords)
                poly_centre = poly.centroid
                location = locator.reverse([poly_centre.x, poly_centre.y])
                place_name = location.address
            elif type(geom) == Marker:
                point_coords = geom.location
                location = locator.reverse(point_coords)
                place_name = location.address
            print(f"{legend_symbol} {place_name}")
chart_selection.observe(on_chart_clicked, 'value')

In [20]:
#v0.6.0: soidgrid data
options_list3 = ['...','Organic carbon density', 'Organic carbon stock', 'Soil organic carbon']
soilgrid_selection = Dropdown(
    options = options_list3,
    description='Dataset'
)
def soilgrid_on_clicked(change):
    with plot_widget:
        plot_widget.clear_output()
        layers = m.ee_layers
        soilgrid_option=change['new']
        soilgrid_layer = ee.Image(veg_indices[soilgrid_option]['collection'])
        sample_scale = m.getScale()
        for geom, ee_object in zip(geom_list, ee_object_list):
            if type(geom) == Polygon:
                latlon = geom.locations
            elif type(geom) == Marker:
                latlon = geom.location
            place_name = None
            if type(geom) == Polygon:
                poly_coords = geom.locations
                poly = shapely.geometry.Polygon(poly_coords)
                poly_centre = poly.centroid
                location = locator.reverse([poly_centre.x, poly_centre.y])
                place_name = location.address
            elif type(geom) == Marker:
                point_coords = geom.location
                location = locator.reverse(point_coords)
                place_name = location.address
            soil_value = soilgrid_layer.reduceRegion(
                reducer=ee.Reducer.mean(),
                geometry = ee_object,
                scale = sample_scale,
                bestEffort = True).getInfo()
            print(f"\U0001F310 {place_name}:")
            if soilgrid_option == 'Organic carbon density':
                print(f"0-5cm mean: {soil_value['ocd_0-5cm_mean']:.2f}")
                print(f"5-15cm mean: {soil_value['ocd_5-15cm_mean']:.2f}")
                print(f"15-30cm mean: {soil_value['ocd_15-30cm_mean']:.2f}")
                print(f"30-60cm mean: {soil_value['ocd_30-60cm_mean']:.2f}")
                print(f"60-100cm mean: {soil_value['ocd_60-100cm_mean']:.2f}")
                print(f"100-200cm mean: {soil_value['ocd_100-200cm_mean']:.2f}")
            elif soilgrid_option == 'Organic carbon stock':
                print(f"0-30cm mean: {soil_value['ocs_0-30cm_mean']:.2f}")
            elif soilgrid_option == 'Soil organic carbon':
                print(f"0-5cm mean: {soil_value['soc_0-5cm_mean']:.2f}")
                print(f"5-15cm mean: {soil_value['soc_5-15cm_mean']:.2f}")
                print(f"15-30cm mean: {soil_value['soc_15-30cm_mean']:.2f}")
                print(f"30-60cm mean: {soil_value['soc_30-60cm_mean']:.2f}")
                print(f"60-100cm mean: {soil_value['soc_60-100cm_mean']:.2f}")
                print(f"100-200cm mean: {soil_value['soc_100-200cm_mean']:.2f}")
soilgrid_selection.observe(soilgrid_on_clicked, 'value')

In [21]:
instructions = VBox(
    [Label('Please, place one or two geometries on the map before selecting a dataset for the chart.'),
     Label('Select either the marker or polygon symbol on the right and draw on the map.')]
)
#Accordion widgets to hold other widgets
menu = Accordion(children=[layers_selection, chart_selection, soilgrid_selection, instructions])
menu.set_title(0, "Layer Selection")
menu.set_title(1, "Time-Series Chart")
menu.set_title(2, "Soilgrid data")
menu.set_title(3, "Help")
chart_control = WidgetControl(widget=menu, position='topright')

In [22]:
# m.add_control(basemap_control)
#ipleaflet function add_control() -> add widgets from ipywidgets to map
m.add_control(dc)
m.add_control(chart_control)
m.add_control(reset_control)
m.add_control(plot_output)

In [23]:
m

Map(center=[52.6, 13.4], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out…